In [ ]:
import bw2data as bd
import bw2calc as bc

In [2]:
bd.projects.set_current("allocation")

In [3]:
if "A" in bd.databases:
    del bd.databases["A"]
    
if "Multifunctional activities" in bd.databases:
    del bd.databases["Multifunctional activities"]

In [4]:
db = bd.Database("A")
db.register()

In [5]:
a = db.new_activity(name="a", code="a", type="emission")
a.save()
# Gets automatic production exchange of amount 1

In [6]:
b = db.new_activity(name="b", code="b", type="process")
b.save()
b.new_exchange(input=a, type="biosphere", amount=5).save()
# Gets automatic production exchange of amount 1

In [7]:
c = db.new_activity(name="c", code="c", type="process")
c.save()
c.new_exchange(input=a, type="biosphere", amount=10).save()
# Gets automatic production exchange of amount 1

In [8]:
d1 = db.new_activity(name="d1", code="d1", type="product") 
d1.save()
d2 = db.new_activity(name="d2", code="d2", type="product") 
d2.save()

I thought we could create a non-`process` activity and it would be ignored, but this isn't how [the current Brightway2 code](https://github.com/brightway-lca/brightway2-data/blob/legacy/bw2data/backends/peewee/database.py#L437) works. This is changed in 2.5. For now we will need to create a sister database for multifunctional activities.

In [9]:
db_mf = bd.Database("Multifunctional activities")
db_mf.register()

In [10]:
# Note: type `multifunctional` is not a thing, just needs not to be `process`
d = db_mf.new_activity(name="d", code="d", type="multifunctional") 
d.save()
d.new_exchange(input=d1, type="production", amount=5, price=10).save()
d.new_exchange(input=d2, type="production", amount=5, price=5).save()
d.new_exchange(input=c, type="technosphere", amount=10).save()
d.new_exchange(input=b, type="technosphere", amount=100).save()

The technosphere matrix won't have `d`:

In [11]:
lca = bc.LCA({c: 1})
lca.lci()
lca.technosphere_matrix.todense()

matrix([[1., 0.],
        [0., 1.]])

Create a function to do allocation:

In [12]:
import bw2data as bd

BUMPER = {'name', 'code'}
SKIPPER = {'input', 'amount', 'output'}


def allocate_database(
    source_database_label: str, 
    target_database_label: str, 
    allocation_type: str = "multifunctional", 
    allocation_attribute: str = "price"
):
    """Take all activities in `Database` `source_database_label` which have the type 
    `allocation_type`, and create allocated copies in `Database` `target_database_label`
    using the attribute `allocation_attribute` as allocation factors.
    
    Appends the suffixes `_0`, `_1`, etc. to the `name` and `code` for the allocated datasets."""
    assert source_database_label in bd.databases
    assert target_database_label in bd.databases
    
    source_db = bd.Database(source_database_label)
    target_db = bd.Database(target_database_label)
    
    count_act, count_exc = 0, 0
    
    for act in source_db:
        if act.get("type") == allocation_type:
            production_exchanges = list(act.production())
            if not production_exchanges:
                print(f"Skipping multifunctional activity {act}, no production exchanges")
                continue
            
            nonproduction_exchanges = list(act.technosphere()) + list(act.biosphere())
            try:
                weights = [exc['amount'] * exc[allocation_attribute] for exc in production_exchanges]
                total = sum(weights)
            except KeyError:
                print(f"Skipping multifunctional activity {act}, missing allocation attribute")
                continue
            
            count_act += 1

            def modifier(key, value, index):
                if key == 'database':
                    return target_database_label
                elif key in BUMPER:
                    return "{}_{}".format(value, index)
                else:
                    return value
                    
            for index, prod_exc in enumerate(production_exchanges):
                node = target_db.new_activity(**{k: modifier(k, v, index) for k, v in act.items()})
                node.save()

                node.new_exchange(
                    input=node,
                    amount=prod_exc['amount'],
                    **{k: v for k, v in prod_exc.items() if k not in SKIPPER}
                ).save()
                
                coefficient = weights[index] / total
                for other_exc in nonproduction_exchanges:
                    node.new_exchange(
                        input=other_exc.input,
                        amount=other_exc['amount'] * coefficient,
                        **{k: v for k, v in other_exc.items() if k not in SKIPPER}
                    ).save()
                    count_exc += 1

    print(f"Created {count_act} activities in {target_database_label} with {count_exc} allocated exchanges")

In [13]:
allocate_database(
    source_database_label="Multifunctional activities", 
    target_database_label="A", 
)

Successfully switch activity dataset to database `A`
Successfully switch activity dataset to database `A`
Created 1 activities in A with 4 allocated exchanges


Check to make sure this did what we expected:

In [14]:
for ds in db:
    if ds['name'].startswith("d_"):
        print(ds)
        for exc in ds.exchanges():
            print(f"\t{exc}")

'd_0' (None, GLO, None)
	Exchange: 5 None 'd_0' (None, GLO, None) to 'd_0' (None, GLO, None)>
	Exchange: 6.666666666666666 None 'c' (None, GLO, None) to 'd_0' (None, GLO, None)>
	Exchange: 66.66666666666666 None 'b' (None, GLO, None) to 'd_0' (None, GLO, None)>
'd_1' (None, GLO, None)
	Exchange: 5 None 'd_1' (None, GLO, None) to 'd_1' (None, GLO, None)>
	Exchange: 3.333333333333333 None 'c' (None, GLO, None) to 'd_1' (None, GLO, None)>
	Exchange: 33.33333333333333 None 'b' (None, GLO, None) to 'd_1' (None, GLO, None)>


And our new technosphere will have `d_0` and `d_1`:

In [15]:
lca = bc.LCA({c: 1})
lca.lci()
lca.technosphere_matrix.todense()

matrix([[  1.        ,   0.        , -66.66666412, -33.33333206],
        [  0.        ,   1.        ,  -6.66666651,  -3.33333325],
        [  0.        ,   0.        ,   5.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   5.        ]])